# 1. Setup: Install Dependencies


In [1]:
pip install tensorflow matplotlib numpy


# 2. Using Pix2Pix (Supervised Model)


**Pix2Pix is ideal for paired datasets, such as edge-to-photo or day-to-night images.**

```Step-by-Step Code for Pix2Pix:```

    Load Dataset: For Pix2Pix, you can use a dataset like facade or maps. Below is an example for loading and preprocessing the Facades dataset.



In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
import os
import matplotlib.pyplot as plt

# Load and preprocess images
def load_image(image_path):
    img = load_img(image_path, target_size=(256, 256))  # Resize to 256x256
    return img_to_array(img) / 127.5 - 1  # Normalize to [-1, 1]

# Load paired dataset (facade dataset example)
def load_dataset(path):
    images = []
    for filename in os.listdir(path):
        img = load_image(os.path.join(path, filename))
        images.append(img)
    return np.array(images)

# Example: Load Facades dataset
real_images = load_dataset('/path/to/real_images')  # Target images (e.g., facade)
input_images = load_dataset('/path/to/input_images')  # Input images (e.g., sketch)


**Define the Pix2Pix Model:**

    Use a U-Net architecture for the Generator.
>
    Use a PatchGAN architecture for the Discrimina

In [ ]:
from tensorflow.keras import layers, Model

# Generator (U-Net)
def build_generator():
    inputs = layers.Input(shape=[256, 256, 3])

    # Downsampling
    down1 = layers.Conv2D(64, 4, strides=2, padding='same')(inputs)
    down1 = layers.LeakyReLU()(down1)

    down2 = layers.Conv2D(128, 4, strides=2, padding='same')(down1)
    down2 = layers.LeakyReLU()(down2)

    # Bottleneck
    bottleneck = layers.Conv2D(256, 4, strides=2, padding='same')(down2)
    bottleneck = layers.LeakyReLU()(bottleneck)

    # Upsampling
    up1 = layers.Conv2DTranspose(128, 4, strides=2, padding='same')(bottleneck)
    up1 = layers.ReLU()(up1)

    up2 = layers.Conv2DTranspose(64, 4, strides=2, padding='same')(up1)
    up2 = layers.ReLU()(up2)

    outputs = layers.Conv2D(3, 4, padding='same', activation='tanh')(up2)

    return Model(inputs, outputs)

# Discriminator (PatchGAN)
def build_discriminator():
    inputs = layers.Input(shape=[256, 256, 3])

    x = layers.Conv2D(64, 4, strides=2, padding='same')(inputs)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2D(128, 4, strides=2, padding='same')(x)
    x = layers.LeakyReLU()(x)

    outputs = layers.Conv2D(1, 4, padding='same')(x)

    return Model(inputs, outputs)

# Build models
generator = build_generator()
discriminator = build_discriminator()


**Compile and Train:**

    Use Binary Crossentropy loss for both generator and discriminator.

In [ ]:
# Compile the discriminator
discriminator.compile(optimizer='adam', loss='binary_crossentropy')

# Compile the generator with a GAN loss (combined generator and discriminator)
generator.compile(optimizer='adam', loss='mae')

# Train the model
for epoch in range(10000):  # Number of epochs
    # Train discriminator with real and fake images
    real = discriminator(real_images)
    fake = discriminator(generator(input_images))

    d_loss_real = discriminator.train_on_batch(real_images, np.ones_like(real))
    d_loss_fake = discriminator.train_on_batch(generator(input_images), np.zeros_like(fake))

    # Train generator to fool the discriminator
    g_loss = generator.train_on_batch(input_images, real_images)  # Try to make fake images look real

    # Print progress
    if epoch % 100 == 0:
        print(f"Epoch {epoch}, D Loss Real: {d_loss_real}, D Loss Fake: {d_loss_fake}, G Loss: {g_loss}")


**Generate Output:**

    Use the trained generator to generate images.

# 3. Using CycleGAN (Unsupervised Model)

    CycleGAN works with unpaired datasets, making it ideal for tasks like horse-to-zebra or photo enhancement.



**Step-by-Step Code for CycleGAN:**

    Load Dataset: Use datasets like horse2zebra or apple2orange.



In [ ]:
import tensorflow as tf
import numpy as np

# Load unpaired dataset
def load_unpaired_images(path):
    images = []
    for filename in os.listdir(path):
        img = load_image(os.path.join(path, filename))  # Load and preprocess image
        images.append(img)
    return np.array(images)

# Load datasets
horse_images = load_unpaired_images('/path/to/horse_images')
zebra_images = load_unpaired_images('/path/to/zebra_images')
